In [69]:
from selenium import webdriver
from time import sleep

from datetime import datetime, timedelta
from geopy import geocoders
from tzwhere import tzwhere
import pytz

tz = tzwhere.tzwhere()

tz_arg = pytz.timezone("America/Argentina/Buenos_Aires")
# tz_rus = pytz.timezone("Europe/Volgograd")
tz_rus = {
    'Moscow': 'Europe/Moscow',
    'Ekaterinburg': 'Asia/Yekaterinburg',
    'St. Petersburg': 'Europe/Moscow',
    'Sochi': 'Europe/Moscow',
    'Kazan': 'Europe/Moscow',
    'Saransk': 'Europe/Moscow',
    'Kaliningrad': 'Europe/Kaliningrad',
    'Samara': 'Europe/Samara',
    'Rostov-On-Don': 'Europe/Moscow',
    'Nizhny Novgorod': 'Europe/Moscow',
    'Volgograd': 'Europe/Volgograd' }

geocoder = geocoders.GoogleV3()  # Para buscar si es que me falta un timezone

In [2]:
d = webdriver.Chrome()
sleep(2)
d.maximize_window()


In [74]:
class Partido:
    def __init__(self, html):
        
        self.link = html.get_attribute('href')
        
        self.grupo = html.find_element_by_class_name("fi__info__group").text
        self.equipos = [x.text for x in html.find_elements_by_class_name("fi-t__nText ")]
        self.ciudad = html.find_element_by_class_name("fi__info__venue").text
        
        # Paso la ciudad a timezone
        if not self.ciudad in tz_rus:
            sleep(2)  # Para no saturar la API
            place, (lat, lng) = geocoder.geocode(self.ciudad)
            tz_rus[self.ciudad] = tz.tzNameAt(lat, lng)
            
        self.timezone = pytz.timezone(tz_rus[self.ciudad])
        # print(self.ciudad, self.timezone)
        
        fecha_aux = html.find_element_by_class_name("fi-mu__info__datetime").text
        self.fecha_local = self.timezone.localize(datetime.strptime(fecha_aux, "%d %b %Y - %H:%M Local time"))
        self.fecha = self.fecha_local.astimezone(tz_arg)
        # print(self.fecha_local, self.fecha)
        
        # No dependo de que actualicen fi-s__scoreText
        if self.fecha + timedelta(minutes=90) <= tz_arg.localize(datetime.now()):
            self.pendiente = False
            try:
                self.marcador = [int(x) for x in html.find_element_by_class_name("fi-s__scoreText").text.split('-')]
            except:
                self.marcador = [ -1, -1 ]
        else:
            self.pendiente = True
        
    def __repr__(self):
        if self.pendiente:
            return "%s [%s] %s - %s" % (self.fecha, self.grupo,
                                          self.equipos[0], self.equipos[1])
        else:
            return "%s [%s] %s %d - %s %d" % (self.fecha, self.grupo,
                                          self.equipos[0], self.marcador[0],
                                          self.equipos[1], self.marcador[1])

In [73]:
d.get('https://www.fifa.com/worldcup/matches/')
fixture = {}

for link in d.find_elements_by_class_name('fi-mu__link'):
    partido = Partido(link)
       
    # Compruebo que el partido esté terminado o en marcha
    if partido.pendiente:
        continue
        
    print(partido)
    
    
    if not partido.grupo in fixture:
        fixture[partido.grupo] = []
        
    fixture[partido.grupo].append(partido)

Moscow Europe/Moscow
2018-06-14 18:00:00+03:00 2018-06-14 12:00:00-03:00
2018-06-14 12:00:00-03:00 [Group A] Russia 5 - Saudi Arabia 0
Ekaterinburg Asia/Yekaterinburg
2018-06-15 17:00:00+05:00 2018-06-15 09:00:00-03:00
2018-06-15 09:00:00-03:00 [Group A] Egypt 0 - Uruguay 1
St. Petersburg Europe/Moscow
2018-06-15 18:00:00+03:00 2018-06-15 12:00:00-03:00
2018-06-15 12:00:00-03:00 [Group B] Morocco 0 - IR Iran 1
Sochi Europe/Moscow
2018-06-15 21:00:00+03:00 2018-06-15 15:00:00-03:00
2018-06-15 15:00:00-03:00 [Group B] Portugal -1 - Spain -1
Kazan Europe/Moscow
2018-06-16 13:00:00+03:00 2018-06-16 07:00:00-03:00
Moscow Europe/Moscow
2018-06-16 16:00:00+03:00 2018-06-16 10:00:00-03:00
Saransk Europe/Moscow
2018-06-16 19:00:00+03:00 2018-06-16 13:00:00-03:00
Kaliningrad Europe/Kaliningrad
2018-06-16 21:00:00+02:00 2018-06-16 16:00:00-03:00
Samara Europe/Samara
2018-06-17 16:00:00+04:00 2018-06-17 09:00:00-03:00
Moscow Europe/Moscow
2018-06-17 18:00:00+03:00 2018-06-17 12:00:00-03:00
Rostov-

In [ ]:
# Busco los goles
for grupo in fixture.keys():
    for partido in fixture[grupo]:
        d.get(partido.link)
        break
    break

In [ ]:
# Terminados
for html_partido in d.find_elements_by_class_name('result'):
    partido = Partido(html_partido)
    print(partido)
    
# Ongoing
for html_partido in d.find_elements_by_class_name('live'):
    partido = Partido(html_partido)
    print(partido)

In [67]:
t = {}
for k in tz_rus:
    t[k] = tz_rus[k].zone

In [68]:
t

{'Moscow': 'Europe/Moscow',
 'Ekaterinburg': 'Asia/Yekaterinburg',
 'St. Petersburg': 'Europe/Moscow',
 'Sochi': 'Europe/Moscow',
 'Kazan': 'Europe/Moscow',
 'Saransk': 'Europe/Moscow',
 'Kaliningrad': 'Europe/Kaliningrad',
 'Samara': 'Europe/Samara',
 'Rostov-On-Don': 'Europe/Moscow',
 'Nizhny Novgorod': 'Europe/Moscow',
 'Volgograd': 'Europe/Volgograd'}